In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os

load_dotenv()

PROJECT_ROOT = os.getenv('PROJECT_ROOT')


In [2]:
import networkx as nx
from tqdm import tqdm
import os

def is_in_europe(lat, lon):
    """
    Checks if a given latitude and longitude fall within an approximate bounding
    box for the European continent.
    
    Europe is approximated as:
      Latitude: 30°N to 72°N
      Longitude: -25°E to 45°E
    """
    return 30 <= lat <= 72 and -25 <= lon <= 45

In [3]:
import random

def generate_random_two_digits():
    return str(random.randint(0, 99)).zfill(2)



In [4]:
def build_graph_from_ats(filename, bidirectional=False):
    """
    Builds a graph from an ATS file containing airway segments.
    
    The ATS file is expected to have header lines for each airway (starting with "A")
    and segment lines (starting with "S"). For example:
    
      A,A1,46
      S,KEC,33.447742,135.794494,ALBAT,33.364503,135.441514,0,262,18.37
      S,ALBAT,33.364503,135.441514,HALON,33.248769,134.997222,260,262,23.34
      ...
      
    For each "S" (segment) line, this function:
      - Parses the start and end fixes and their coordinates.
      - Filters out segments if either endpoint is not within Europe.
      - Adds fixes (nodes) to the graph.
      - Adds an edge for the segment with attributes for minimum altitude,
        maximum altitude, distance, and the airway name.
    
    Parameters:
      filename (str): Path to the ATS data file.
      bidirectional (bool): If True, add edges in both directions.
    
    Returns:
      networkx.DiGraph: The directed graph representing ATS fixes and segments.
    """
    # Create a directed graph; convert later to undirected if needed.
    G = nx.DiGraph()
    current_airway = "Unknown"  # Default airway name if header hasn't been seen

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue  # Skip empty or comment lines

            parts = line.split(',')
            record_type = parts[0].strip().upper()

            # If the line is an airway header, update the current airway name.
            if record_type == "A":
                if len(parts) >= 2:
                    current_airway = parts[1].strip()
                    # Append two random digits to the airway name to make it unique
                    current_airway = current_airway + generate_random_two_digits()
                continue


            # Process only segment lines.
            if record_type != "S":
                continue

            if len(parts) < 10:
                print(f"Skipping malformed line: {line}")
                continue

            # Unpack fields from the segment line.
            # Format: S, start_fix, start_lat, start_lon, end_fix, end_lat, end_lon, min_alt, max_alt, distance
            (_, start_fix, start_lat, start_lon, end_fix, end_lat, end_lon,
             min_alt, max_alt, distance) = parts
            
            # Append two random digits to the start and end fixes to make them unique
            start_fix = start_fix + generate_random_two_digits()
            end_fix = end_fix + generate_random_two_digits()

            # Convert coordinate and numeric fields.
            try:
                start_lat = float(start_lat)
                start_lon = float(start_lon)
                end_lat   = float(end_lat)
                end_lon   = float(end_lon)
                min_alt   = float(min_alt)
                max_alt   = float(max_alt)
                distance  = float(distance)
            except ValueError:
                print(f"Skipping line due to conversion error: {line}")
                continue

            # Filter: Only include segments with both endpoints in Europe.
            if not (is_in_europe(start_lat, start_lon) and is_in_europe(end_lat, end_lon)):
                # Uncomment the next line to see which segments are skipped.
                # print(f"Skipping non-European segment: {start_fix} -> {end_fix}")
                continue

            # Add the nodes (fixes) with their coordinates.
            if start_fix not in G:
                G.add_node(start_fix, lat=start_lat, lon=start_lon)
            if end_fix not in G:
                G.add_node(end_fix, lat=end_lat, lon=end_lon)

            # Add the edge (segment) with its attributes, including the airway name.
            edge_attrs = {
                'min_alt': min_alt,
                'max_alt': max_alt,
                'distance': distance,
                'airway': current_airway,
                'edge_type': 'airway'
            }
            G.add_edge(start_fix, end_fix, **edge_attrs)


            # Optionally, add the reverse edge if the segment is bidirectional.
            if bidirectional:
                G.add_edge(end_fix, start_fix, **edge_attrs)

    return G

In [5]:
def build_graph_from_waypoints(filename):
    """
    Builds a graph from a Waypoints file containing fix positions.
    
    The Waypoints file is expected to have lines with the format:
    fix_name,latitude,longitude,unused_field
    
    For example:
      0000E,0.000000,0.000000,  
      0000N,0.000000,0.000000,  
    
    For each line, this function:
      - Parses the fix name and its coordinates
      - Filters out fixes that are not within Europe
      - Adds the fix as a node to the graph with its coordinates
    
    Parameters:
      filename (str): Path to the Waypoints data file.
    
    Returns:
      networkx.Graph: The undirected graph containing waypoint nodes.
    """
    # Create an undirected graph since we're only adding nodes
    G = nx.Graph()

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue  # Skip empty or comment lines

            parts = line.split(',')
            if len(parts) < 3:
                print(f"Skipping malformed line: {line}")
                continue

            # Unpack fields from the line
            fix_name, lat, lon = parts[:3]
            
            # Append two random digits to the fix name to make it unique
            fix_name = fix_name + generate_random_two_digits()

            # Convert coordinate fields
            try:
                lat = float(lat)
                lon = float(lon)
            except ValueError:
                print(f"Skipping line due to conversion error: {line}")
                continue

            # Filter: Only include fixes in Europe
            if not is_in_europe(lat, lon):
                # Uncomment the next line to see which fixes are skipped
                # print(f"Skipping non-European fix: {fix_name}")
                continue

            # Add the node (fix) with its coordinates
            G.add_node(fix_name, lat=lat, lon=lon)

    return G

In [6]:
ats_file = os.path.join(PROJECT_ROOT, "data", "airac", "ATS.txt")

# Set bidirectional=True if segments are used in both directions.
graph_ats = build_graph_from_ats(ats_file, bidirectional=False)

# Define the output path for the graph file
graph_output_path = os.path.join(PROJECT_ROOT, "data", "graphs", "ats_graph.graphml")

# Ensure the directory exists
os.makedirs(os.path.dirname(graph_output_path), exist_ok=True)

# Save the graph in GraphML format
nx.write_graphml(graph_ats, graph_output_path)
print(f"ATS graph saved to: {graph_output_path}")


print(f"Number of fixes (nodes): {graph_ats.number_of_nodes()}")
print(f"Number of route segments (edges): {graph_ats.number_of_edges()}")

print('--------------------------------')

# Build the waypoints graph
waypoints_file = os.path.join(PROJECT_ROOT, "data", "airac", "WAYPOINTS.txt")
graph_waypoints = build_graph_from_waypoints(waypoints_file)

# Define the output path for the waypoints graph file
waypoints_output_path = os.path.join(PROJECT_ROOT, "data", "graphs", "waypoints_graph.graphml")

# Ensure the directory exists
os.makedirs(os.path.dirname(waypoints_output_path), exist_ok=True)

# Save the waypoints graph in GraphML format
nx.write_graphml(graph_waypoints, waypoints_output_path)
print(f"Waypoints graph saved to: {waypoints_output_path}")
print(f"Number of fixes (nodes): {graph_waypoints.number_of_nodes()}")
print('--------------------------------')

ATS graph saved to: E:/project-akrav\data\graphs\ats_graph.graphml
Number of fixes (nodes): 54983
Number of route segments (edges): 28950
--------------------------------
Waypoints graph saved to: E:/project-akrav\data\graphs\waypoints_graph.graphml
Number of fixes (nodes): 59362
--------------------------------


In [7]:
def group_waypoints(graph_waypoints):
    """
    Groups waypoints that are within 10 nautical miles of each other using a grid-based approach.
    
    Parameters:
        graph_waypoints (networkx.Graph): Graph containing waypoints as nodes with lat/lon attributes
        
    Returns:
        dict: Mapping of original waypoint names to their group IDs
    """
    from math import radians, cos, sin, asin, sqrt
    import numpy as np
    
    def haversine(lat1, lon1, lat2, lon2):
        """Calculate haversine distance in nautical miles"""
        R = 3440.065  # Earth's radius in nautical miles
        
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        return R * c
    
    # Convert 10 nautical miles to approximate degrees at the equator
    # (1 degree ≈ 60 nautical miles)
    grid_size = 10/60  # degrees
    
    # Initialize grid cells dictionary
    grid_cells = {}
    waypoint_groups = {}
    current_group = 0
    
    # First pass: Assign waypoints to grid cells
    for node in graph_waypoints.nodes():
        lat = graph_waypoints.nodes[node]['lat']
        lon = graph_waypoints.nodes[node]['lon']
        
        # Calculate grid cell indices
        cell_lat = int(lat / grid_size)
        cell_lon = int(lon / grid_size)
        
        # Store waypoint in grid cell
        cell_key = (cell_lat, cell_lon)
        if cell_key not in grid_cells:
            grid_cells[cell_key] = []
        grid_cells[cell_key].append((node, lat, lon))
    
    # Second pass: Group waypoints
    processed = set()
    
    for cell_key, waypoints in grid_cells.items():
        # Get neighboring cells
        cell_lat, cell_lon = cell_key
        neighbor_cells = [
            (cell_lat + i, cell_lon + j)
            for i in [-1, 0, 1]
            for j in [-1, 0, 1]
        ]
        
        for waypoint, lat1, lon1 in waypoints:
            if waypoint in processed:
                continue
                
            # Start a new group
            current_group_waypoints = []
            
            # Check waypoints in current and neighboring cells
            for neighbor_cell in neighbor_cells:
                if neighbor_cell not in grid_cells:
                    continue
                    
                for other_waypoint, lat2, lon2 in grid_cells[neighbor_cell]:
                    if other_waypoint in processed:
                        continue
                        
                    if haversine(lat1, lon1, lat2, lon2) <= 10:
                        current_group_waypoints.append(other_waypoint)
                        processed.add(other_waypoint)
            
            if current_group_waypoints:
                for wp in current_group_waypoints:
                    waypoint_groups[wp] = current_group
                current_group += 1
            
            processed.add(waypoint)
    
    # Handle any remaining unprocessed waypoints
    for node in graph_waypoints.nodes():
        if node not in waypoint_groups:
            waypoint_groups[node] = current_group
            current_group += 1
    
    return waypoint_groups

waypoint_groups = group_waypoints(graph_waypoints)


In [8]:
def reduce_graph_by_groups(graph, waypoint_groups):
    """
    Reduces the graph by selecting one representative node per group.
    
    Parameters:
        graph (networkx.Graph): Original graph containing waypoints
        waypoint_groups (dict): Mapping of waypoint names to their group IDs
        
    Returns:
        networkx.Graph: Reduced graph with one waypoint per group
    """
    import networkx as nx
    
    # Create a new graph for the reduced version
    reduced_graph = nx.Graph()
    
    # Create reverse mapping from group ID to list of waypoints
    group_to_waypoints = {}
    for waypoint, group_id in waypoint_groups.items():
        if group_id not in group_to_waypoints:
            group_to_waypoints[group_id] = []
        group_to_waypoints[group_id].append(waypoint)
    
    # For each group, select a representative waypoint
    # (here we choose the first waypoint in each group)
    for group_id, waypoints in group_to_waypoints.items():
        representative = waypoints[0]
        # Copy the node and its attributes to the reduced graph
        reduced_graph.add_node(representative, **graph.nodes[representative])
    
    return reduced_graph

# Use the function
reduced_waypoints_graph = reduce_graph_by_groups(graph_waypoints, waypoint_groups)

print(f"Original graph nodes: {graph_waypoints.number_of_nodes()}")
print(f"Reduced graph nodes: {reduced_waypoints_graph.number_of_nodes()}")

# Save the reduced graph
reduced_output_path = os.path.join(PROJECT_ROOT, "data", "graphs", "reduced_waypoints_graph.graphml")
nx.write_graphml(reduced_waypoints_graph, reduced_output_path)
print(f"Reduced waypoints graph saved to: {reduced_output_path}")

Original graph nodes: 59362
Reduced graph nodes: 13698
Reduced waypoints graph saved to: E:/project-akrav\data\graphs\reduced_waypoints_graph.graphml


In [9]:
import networkx as nx
from geopy.distance import geodesic
from sklearn.neighbors import BallTree
import numpy as np
from tqdm import tqdm

def merge_graphs_balltree(graph_ats, reduced_waypoints_graph,
                          max_dct_distance):
    """
    Merge the ATS (airway) graph with the waypoint graph and add potential DCT (direct) edges.
    Uses a BallTree for efficiently finding nodes that are close enough.
    
    Parameters:
      graph_ats: networkx DiGraph with nodes and published airway edges.
      reduced_waypoints_graph: networkx Graph (or DiGraph) with additional waypoint nodes.
      ct: time cost (currency per hour) for cost computation.
      cf: fuel cost (currency per kg) for cost computation.
      max_dct_distance: maximum distance (nm) within which to add a DCT edge.
      
      
    Returns:
      G_merged: A networkx DiGraph that includes:
          - All nodes from both graphs.
          - All original ATS (airway) edges.
          - Additional DCT edges (with attributes) connecting nodes that are within
            max_dct_distance and are not already connected by an airway edge.
    """
    
    # --- Step 1. Merge nodes and ATS edges into a new graph ---
    G_merged = nx.DiGraph()
    
    # Add ATS nodes and edges
    for node, data in graph_ats.nodes(data=True):
        G_merged.add_node(node, **data)
    for u, v, data in graph_ats.edges(data=True):
        G_merged.add_edge(u, v, **data)
    
    # Add waypoint nodes that are not already present
    for node, data in reduced_waypoints_graph.nodes(data=True):
        if node not in G_merged:
            G_merged.add_node(node, **data)
    
    # --- Step 2. Build a BallTree of nodes with valid coordinates ---
    # Create a list of nodes (identifiers) and a corresponding coordinate array in radians.
    all_nodes = list(G_merged.nodes())
    coords = []
    valid_node_indices = []  # indices for which we have valid coordinates
    valid_nodes = []         # corresponding node ids
    
    for idx, node in enumerate(all_nodes):
        data = G_merged.nodes[node]
        lat = data.get('lat')
        lon = data.get('lon')
        if lat is None or lon is None:
            # Skip nodes without valid coordinate data
            coords.append((np.nan, np.nan))
        else:
            # Convert to radians: [latitude, longitude]
            coords.append((np.radians(lat), np.radians(lon)))
            valid_node_indices.append(idx)
            valid_nodes.append(node)
    
    coords = np.array(coords)
    
    # Filter out nodes with missing coordinates for building the tree
    valid_coords = coords[valid_node_indices]
    if len(valid_coords) == 0:
        # No valid nodes: nothing to add.
        return G_merged

    # Build the BallTree using haversine metric.
    ball_tree = BallTree(valid_coords, metric='haversine')
    
    # --- Step 3. Query the BallTree to add potential DCT edges ---
    # Earth radius in nautical miles (approximate)
    earth_radius_nm = 3440.065
    # Convert max_dct_distance (nm) to radians.
    radius_radians = max_dct_distance / earth_radius_nm
    
    # Query: for each valid node, find neighbors within the specified radius.
    # query_radius returns an array (one per point) of indices into valid_coords.
    neighbors_indices = ball_tree.query_radius(valid_coords, r=radius_radians)
    
    # Loop over each valid node and its neighbors
    for i, neighbor_idxs in tqdm(enumerate(neighbors_indices), total=len(neighbors_indices)):
        node_u = valid_nodes[i]
        data_u = G_merged.nodes[node_u]
        lat_u = data_u.get('lat')
        lon_u = data_u.get('lon')
        for j in neighbor_idxs:

            if i == j:
                continue  # Skip self
            node_v = valid_nodes[j]
            # If there is already an edge from node_u to node_v (an ATS edge), skip.
            if G_merged.has_edge(node_u, node_v):
                continue
            data_v = G_merged.nodes[node_v]
            lat_v = data_v.get('lat')
            lon_v = data_v.get('lon')
            # Compute the geodesic distance in nautical miles.
            distance = geodesic((lat_u, lon_u), (lat_v, lon_v)).nautical
            if distance > max_dct_distance:
                continue  # Skip if outside desired threshold (can happen near the boundary)
            
            # Create the DCT edge data.
            dct_edge_data = {
                'distance': distance,
                'min_alt': 0,
                'max_alt': 0,
                'airway': '',
                'edge_type': 'DCT'  # mark the edge as a direct (DCT) edge
            }
            G_merged.add_edge(node_u, node_v, **dct_edge_data)
            G_merged.add_edge(node_v, node_u, **dct_edge_data)

    

    return G_merged

merged_graph = merge_graphs_balltree(graph_ats, reduced_waypoints_graph, 40)
# Write the merged graph to a file
nx.write_gml(merged_graph, os.path.join(PROJECT_ROOT, "data", "graphs", "route_graph.gml"))


100%|██████████| 68424/68424 [11:36<00:00, 98.26it/s]  


In [10]:
def collapse_duplicate_nodes(graph):
    """
    Collapses nodes that are extremely close together (distance < 0.001 nm).
    When nodes are collapsed, one node is kept and the other is removed, with all edges
    redirected to the remaining node.
    
    Args:
        graph: NetworkX graph to process
        
    Returns:
        Modified graph with duplicate nodes collapsed
    """
    G = graph.copy()
    
    # Find pairs of nodes to collapse
    nodes_to_merge = []
    for u, v, data in G.edges(data=True):
        if data['distance'] < 0.001:
            nodes_to_merge.append((u, v))

    nodes_removed = 0
    print(f'Found {len(nodes_to_merge)} duplicate nodes')
    
    # For each pair of nodes to merge
    for node1, node2 in tqdm(nodes_to_merge, total=len(nodes_to_merge)):
        if node1 not in G.nodes() or node2 not in G.nodes():
            continue # Skip if either node was already removed
            

        # Get all edges connected to node2
        edges_to_redirect = list(G.in_edges(node2, data=True)) + list(G.out_edges(node2, data=True))
        
        # Redirect all edges from/to node2 to instead connect with node1
        for u, v, data in edges_to_redirect:
            if u == node2:
                if v != node1:  # Avoid self loops
                    G.add_edge(node1, v, **data)
            elif v == node2:
                if u != node1:  # Avoid self loops
                    G.add_edge(u, node1, **data)
                    
        # Remove the duplicate node
        G.remove_node(node2)
        nodes_removed += 1

    print(f"Removed {nodes_removed} duplicate nodes")

    return G

collapsed_graph = collapse_duplicate_nodes(merged_graph)


Found 539412 duplicate nodes


100%|██████████| 539412/539412 [00:25<00:00, 21060.11it/s] 

Removed 49637 duplicate nodes


In [11]:
print(f"Merged graph nodes: {collapsed_graph.number_of_nodes()}")
print(f"Merged graph edges: {collapsed_graph.number_of_edges()}")




Merged graph nodes: 18787
Merged graph edges: 513277


In [12]:
# Write the merged graph to a file
nx.write_gml(collapsed_graph, os.path.join(PROJECT_ROOT, "data", "graphs", "route_graph_reduced.gml"))





In [13]:
def generate_airport_csv(filename):
    """
    Extracts airport information from Airport.txt and saves it to a CSV file.
    
    Airport line format:
    A,ICAO,NAME,LAT,LON,ELEV,LENGTH,UNUSED1,UNUSED2,UNUSED3
    
    Parameters:
        filename (str): Path to the Airport.txt file
    """
    import csv
    import os
    
    output_file = os.path.join(os.path.dirname(filename), 'airports.csv')
    
    # Define CSV headers
    headers = ['icao', 'name', 'latitude', 'longitude', 'elevation']
    
    with open(filename, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(headers)
        
        for line in infile:
            line = line.strip()
            if not line or not line.startswith('A'):
                continue
                
            parts = line.split(',')
            if len(parts) < 5:
                continue
                
            # Extract relevant fields
            icao = parts[1]
            name = parts[2]
            try:
                lat = float(parts[3])
                lon = float(parts[4])
                elev = float(parts[5])
            except (ValueError, IndexError):
                print(f"Skipping malformed line: {line}")
                continue
                
            writer.writerow([icao, name, lat, lon, elev])
    
    print(f"Airport data saved to: {output_file}")

# Usage:
airport_file = os.path.join(PROJECT_ROOT, "data", "airac", "Airports.txt")
generate_airport_csv(airport_file)

Airport data saved to: E:/project-akrav\data\airac\airports.csv


In [16]:
# Find node HELEN13 in the merged_graph
merged_graph.nodes['HELEN13']

{'lat': 51.235314, 'lon': 3.869711}

In [17]:
collapsed_graph.nodes['HELEN13']

{'lat': 51.235314, 'lon': 3.869711}